# Simplex

#### Katherine Yohanna Mazariegos Guerra
#### Abner Xocop Chacach

In [1]:
import numpy as np
import sys
import itertools

In [2]:
# En archivo exercise01.txt x1, x2 y x3 es el nombre de las variables, se usaron esas para seguir con el estándar de nombramiento en Programación Lineal.
# Es necesario escribir si es min/max para que el programa identifique si es de maximización o minimización, st para indicar que esas son las restricciones y end para indicar al programa que el problema termina ahí

def parse_coefficients(coefficient_list, monomial):
    """
    Este es un parseador de coeficientes. Consiste en comprobar si una cadena tiene una expresión regular en donde pueda extraer caracteres específicos, en este caso se busca extraer los coeficientes.

    Args:
        :rtype: None
        :param coefficient_list: Lista en la que se almacenarán los coeficientes
        :param monomial: Una cadena (por ejemplo, -3x1) que será analizada hasta su coeficiente (por ejemplo, -3)

    Verifica qué patrón coincide. Válidos son: (s)(n)lv
        Los paréntesis indican la existencia opcional
        s es + o - (la ausencia significa +)
        n es un número (coeficiente, la ausencia significa 1)
        l es una letra latina minúscula (letra variable)
        v es un número, probablemente incremental (número variable)

    Import re:
        Una expresión regular (o RE) especifica un conjunto de cadenas que se corresponde con ella; las funciones de este módulo le permiten comprobar si una cadena particular se corresponde con una expresión regular dada (o si una expresión regular dada se corresponde con una cadena particular, que se reduce a lo mismo)
        Source: https://docs.python.org/3/library/re.html
    """
    import re

    if re.match('[ ]*[\+ ]?[\d]+[\.]?[\d]*', monomial):
        float_cast = float(re.match('[ ]*[\+ ]?[\d]+[\.]?[\d]*', monomial).group(0))
        coefficient_list.append(float_cast)
    elif re.match('[ ]*[\-][\d]+[\.]?[\d]*', monomial):
        float_cast = float(re.match('[ ]*[\-][\d]+[\.]?[\d]*', monomial).group(0))
        coefficient_list.append(float_cast)
    elif re.match('[ ]*[\+]*[a-z][\d]+', monomial):
        coefficient_list.append(1)
    elif re.match('[ ]*[\-][a-z][\d]+', monomial):
        coefficient_list.append(-1)

In [3]:
lines = []
def parse_lp1(input_filename):
    """
    Esta función es la encargada de leer el archivo, hará uso del parser para mandar línea a línea el contenido del .txt y según los coeficientes que obtenga devolverá las matrices y arrays correspondientes. 

    Tiene tareas como verificar que el archivo haya sido encontrado, leer si es un problema de maximización o minimización, llenar las matrices/ arrays.

    :rtype : tuple
    :param input_filename: Nombre de archivo de la entrada del problema lineal
    :return: Retorna A-matrix, b-vector, c-vector, MinMax
    """
    import re

    error = 0 # Inicializar la variable de error. Si el error!=0 entonces hubo un problema de entrada de archivo 


    try:
        infile = open('Simplex.txt')
    except FileNotFoundError:
        error = 1
        print('\nInput file error: Archivo no encontrado.') # Archivo no encontrado

    #lines = []
    if error != 1:
        for line in infile:
            lines.append(line)
        infile.close()
    for line in lines:
        print(line, end='')    

    minmax_line = '' # Verficar si el problema es de maximización o de minimización
    for line in lines:
        if re.match('^[ ]*max|min', line):
            minmax_line = line

    minmax = 0
    objective_function = ''

    if re.match('^[ ]*max', minmax_line): #Si en el archivo se encuentra la palabra 'max' entonces el problema es de maximización
        minmax = -1
        objective_function = minmax_line
        objective_function = objective_function.strip('max')
    
    elif re.match('^[ ]*min', minmax_line): # Si en el archivo se encuentra la palabra 'min' entonces el problema es de minimización
        minmax = 1
        objective_function = minmax_line
        objective_function = objective_function.strip('min')

    if minmax_line == '' and minmax == 0: # Si en el archivo no se encuentra ni 'max' ni  'min' entonces no hay función objetivo
        error = 2
        print('\nInput file error: Función objetivo no encontrada.')

    c_vector = [] # Rellenar el vector c con coeficientes de función objetiva

    regex = re.compile('^[\+\- ]?[\d]*[\.]?[\d]*[a-z][\d+]')
    while regex.match(objective_function):
        monomial = regex.match(objective_function).group(0)
        parse_coefficients(c_vector, monomial)
        objective_function = objective_function.replace(monomial, '', 1)

    a_matrix = [] # Rellenar la matriz A (coeficientes) y el vector b utilizando las restricciones del problema
    b_vector = []
    eqin = []

    st_line = ''
    st_index = 0
    for index, line in enumerate(lines):
        if 'st' in line:
            st_index = index
            st_line = line

    if re.match('^[ ]*st', st_line):
        st_line = st_line.replace('st', '  ', 1)

    if st_line == '':
        error = 3
        print('\nInput file error: Línea de restricciones no encontrada. No existe la keyword \'st\'.')

    while st_index < len(lines) - 1:
        sub_a_vector = []
        a_matrix.append(sub_a_vector)
        while True:
            st_line = st_line.strip(' ')
            if re.match('^[\+\- ]?[\d]*[\.]?[\d]*[a-z][\d+]', st_line):
                monomial = re.match('^[\+\- ]?[\d]*[\.]?[\d]*[a-z][\d+]', st_line).group(0)
                parse_coefficients(sub_a_vector, monomial)
                st_line = st_line.replace(monomial, '', 1)
            elif re.match('^[<>=]+', st_line):
                monomial = re.match('^[<>=]+', st_line).group(0)
                if monomial == '<=':
                    eqin.append(-1)
                elif monomial == '>=':
                    eqin.append(1)
                elif monomial == '==':
                    eqin.append(0)
                else:
                    error = 4
                    print('\nInput file error: Caracter inesperado; esperados <=, >=, = al menos', monomial)
                st_line = st_line.replace(monomial, '', 1)
            elif re.match('^[\d]+', st_line):
                monomial = re.match('^[\d]+', st_line).group(0)
                int_cast = int(re.match('^[\d]+', st_line).group(0))
                b_vector.append(int_cast)
                st_line = st_line.replace(monomial, '', 1)
            else:
                if not sub_a_vector:    # Evalúa true cuando las líneas están vacías entre las restricciones
                    a_matrix.pop()
                break

        st_index += 1 #         Incrementar el número de línea y obtener el siguiente
        st_line = lines[st_index]

        if st_line == 'end\n' and error == 0: #         Búsqueda de la declaración final y ausencia de errores
            print('\nArchivo cargado exitosamente.')
            break

    return a_matrix, b_vector, c_vector, eqin, minmax # Devolver todas las listas y variables creadas

In [4]:
def convert_to_dual(input_filename, output_filename):
    """
    Verifica si son restricciones de >=, <= o =. También tiene como tarea hacer un archivo de salida en el que muestre los resultados de las matrices que se llenaron.

    :param input_filename: Nombre de archivo de la entrada del problema lineal
    :param output_filename: Filename of the linear problem output
    :return: Returns A-matrix, b-vector, c-vector, Variable-constraints, MinMax
    """

    (a_matrix, b_vector, c_vector, eqin, minmax) = parse_lp1(input_filename)     # Llamar la función parse_lp1

    variable_constraints = [] # Convertir las restricciones a equivalentes duales '*' significa libre
    if minmax == -1:
        for el in eqin:
            if el == 0:
                variable_constraints.append('==')
            elif el == 1:
                variable_constraints.append('>=')
            elif el == -1:
                variable_constraints.append('<=')

    a_matrix = list(zip(a_matrix)) # Traspuesta de A-matrix

    minmax = -minmax # min(max) el problema dual es max(min)

    outfile = open(output_filename, 'w') # Escribir el problema a un archivo de salida
    outfile.write('(Objective Function) b-vector: [' + ', '.join(map(str, b_vector)) + ']\n')

    outfile.write('\nA-matrix: [')
    thing = ''
    for index, sub_a_vector in enumerate(a_matrix):
        thing += '[ ' + ', '.join(map(str, sub_a_vector)) + ']'
        if index != (len(a_matrix) - 1):
            thing += ', '
    outfile.write(thing + ']\n')

    outfile.write('\n(Contraints) c-vector: [' + ', '.join(map(str, c_vector)) + ']\n')
    outfile.write('\n(Variable Contraints) variable_constraints-vector: [' + ', '.join(map(str, c_vector)) + ']\n')
    outfile.write('\nEqin: [' + ', '.join(map(str, eqin)) + ']\n')
    outfile.write('\nMinMax: [' + str(minmax) + ']\n')
    outfile.close()

    return a_matrix, b_vector, c_vector, variable_constraints, eqin, minmax

(a_matrix, b_vector, c_vector, variable_contraints, eqin, minmax) = convert_to_dual('input-lp1', 'output-lp2')


max 10x1+15x2
st 282x1+400x2<=2000
   4x1+40x2<=140
   1x1+0x2<=5
end

In [5]:
"""
únicamente imprimimos los distintos arrays necesarios para realizar el programa.
"""

print(a_matrix)
print(b_vector)
print(c_vector)
print(variable_contraints)

[([282.0, 400.0],), ([4.0, 40.0],), ([1.0, 0.0],)]
[2000, 140, 5]
[10.0, 15.0]
['<=', '<=', '<=']


In [6]:
"""
Solicita el número de restricciones para que el programa sepa las iteraciones que tiene que hacer para 
la matriz de restricciones
"""
no_restricciones = int(input("Ingrese el no. de restricciones: "))

Ingrese el no. de restricciones: 3


In [7]:
"""
Solicita el número de variables para que sepa la cantidad de columnas que tiene que tener el programa para crear una matriz
de restricciones.

Variables sirven para saber el número de columnas de la matriz
Restricciones sirven para saber el número de filas de la matriz
"""

no_variables = int(input("Ingrese el no. de variables: "))

Ingrese el no. de variables: 2


In [8]:
'''
Revisar si los coeficientes son binarios o no.

Lo que es hace es multiplicar la cantidad de variables por la cantidad de restricciones, esto indicará la cantidad
de coeficientes que estarán en la matriz. 

Se añadió un contador llamado "sumador_binarios", que se autosuma una unidad cuando encuentra una variable binaria.

Cuando el número de coeficientes coincida con la suma del contador, es porque ya puede resolver la maximización.
'''
sumador_enteros = 0
numero_para_cant_enteros = no_variables * no_restricciones
enteros = 0

a_matrix2 = []

for i in a_matrix:
    a_matrix2.append((i[0]))
print(a_matrix2)

for row in a_matrix2:
       for elem in row:
            num_int = round(elem)
            #check_int = isinstance(elem, int)
            if elem-num_int != 0:                
                print(elem)
                print("No todos los coeficientes son enteros, vuelva ingresar")
            else:
                sumador_enteros+=1
                print(elem)
                print("Coeficiente sí es entero")
                enteros = 1
                
print("numero_para_cant_enteros ", numero_para_cant_enteros)
print("sumador_enteros", sumador_enteros)

if sumador_enteros == numero_para_cant_enteros:
    print("Ya todos son enteros, se resolverá la maximización") 
else: 
    print("Debe actualizar para que sean sólo enteros")
    sys.exit(1)
    
      

[[282.0, 400.0], [4.0, 40.0], [1.0, 0.0]]
282.0
Coeficiente sí es entero
400.0
Coeficiente sí es entero
4.0
Coeficiente sí es entero
40.0
Coeficiente sí es entero
1.0
Coeficiente sí es entero
0.0
Coeficiente sí es entero
numero_para_cant_enteros  6
sumador_enteros 6
Ya todos son enteros, se resolverá la maximización


In [9]:
'''
Revisar si los coeficientes de la matriz C son binarios o no.

Lo que es hace es tomar el cuenta el número de variables.. 

Se añadió un contador llamado "sumador_binarios_c", que se autosuma una unidad cuando encuentra una variable binaria.

Cuando el número de variables coincida con la suma del contador, es porque ya puede resolver la maximización.
'''

print(c_vector)

sumador_enteros_c = 0
numero_para_cant_enteros_c = no_variables




for row in c_vector:
    num_int = round(elem)
    #check_int = isinstance(elem, int)
    if elem-num_int != 0:
        print(elem)
        print("No todos los coeficientes son enteros, vuelva ingresar")
    else:
        sumador_enteros_c+=1
        print(elem)
        print("Coeficiente sí es entero")
        enteros = 1
        
print("numero_para_cant_enteros ", numero_para_cant_enteros_c)
print("sumador_enteros", sumador_enteros_c)

if sumador_enteros_c == numero_para_cant_enteros_c:
    print("Ya todos son enteros, se resolverá la maximización") 
else: 
    print("Debe actualizar para que sean sólo enteros")
    sys.exit(1)

[10.0, 15.0]
0.0
Coeficiente sí es entero
0.0
Coeficiente sí es entero
numero_para_cant_enteros  2
sumador_enteros 2
Ya todos son enteros, se resolverá la maximización


In [10]:
#Hacer matriz
#aqui estan los coeficientes de las restricciones
"""
Aqui se hace la matriz con las slack variables y con las variables artificiales que sean necesarias
"""
positions = []
a_matrix2 = []

for i in a_matrix:
    a_matrix2.append((i[0]))
#print(a_matrix2[0])
print(a_matrix2)
#convertimos a_matrix2 en una matriz de numpy para hacer operaciones con ella
mat = np.asmatrix(a_matrix2)
size = len(variable_contraints)
for i in range (len(variable_contraints)):
    if variable_contraints[i] == "<=":
        #hacemos la columna de ceros, de tamaño igual 
        # a las columnas que están en la matrix (length del arreglo de símbolos)
        new_matrix = np.asmatrix(np.zeros(size))
        #colocamos 1 en el espacio correspondiente de la fila en donde está 
        # la constrait
        new_matrix[0,i]=1
        #asignamos la forma para que sea un vector columna
        new_matrix.shape = (size,1)
        # obtenemos las dimensiones de la matrix actual
        x, y = mat.shape
        # hacemos el append de la columna de ceros y el uno en la posición correspondiente
        # al final de la matriz, a la derecha
        mat = np.hstack((mat[:,:y], new_matrix, mat[:,y:]))        
        print(a_matrix2[i])
        print("menor")
    if variable_contraints[i] == "==":
        new_matrix = np.asmatrix(np.zeros(size))
        new_matrix[0,i]=1
        new_matrix.shape = (size,1)
        x, y = mat.shape
        mat = np.hstack((mat[:,:y], new_matrix, mat[:,y:]))  
        print(a_matrix2[i])
        print("igual")
        positions.append(y)
    if variable_contraints[i] == ">=":
        new_matrix = np.asmatrix(np.zeros(size))
        new_matrix[0,i]=1
        new_matrix1 = np.asmatrix(np.zeros(size))
        new_matrix1[0,i]=-1
        new_matrix.shape = (size,1)
        new_matrix1.shape = (size,1)
        x, y = mat.shape
        mat = np.hstack((mat[:,:y], new_matrix, mat[:,y:]))
        mat = np.hstack((mat[:,:y+1], new_matrix1, mat[:,y+1:]))
        print(a_matrix2[i])
        print("mayor")
        positions.append(y)
    #print(variable_contraints[i])
#print(variable_contraints[0])
print(mat)

#Numero de columnas en la matriz
num_cols = mat.shape[1]
print(num_cols)
print(positions)

[[282.0, 400.0], [4.0, 40.0], [1.0, 0.0]]
[282.0, 400.0]
menor
[4.0, 40.0]
menor
[1.0, 0.0]
menor
[[282. 400.   1.   0.   0.]
 [  4.  40.   0.   1.   0.]
 [  1.   0.   0.   0.   1.]]
5
[]


In [11]:
"""
Aquí es la orquetastación principal, porque aquí es donde se miden el número de filas y columnas que tendrá la tableau.
Así mismo se inicializa el algoritmo simplex de manera explícita. También se le indica si es un problema tipo Min o Max.Luego
empieza a buscar los puntos pivote, apoyándose de la eliminación gaussiana.

Más adelante han sido definidas dos funciones: una de maximización, donde se le envía la palabra 'MAX' y esta función
reconoce que tiene que resolver una maximización. La otra función es de minimización, que envía la palabra 'MIN'
lo que significa que se requiere de una minimización.

Estas palabras permiten que el algoritmo prepare la tableau según lo requiere la maximización y la minimización, porque 
ambos tienen una resolución distinta.
"""

def simplex(of,basis,tableau,opt):
    # Obtiene el número de filas y columnas que tiene la tableau
    n_rows = tableau.shape[0]
    n_cols = tableau.shape[1]
    if opt =='MIN':
        # Inicia el algoritmo simplex
        # Calcula zj-cj. Si cj - zj >= 0 para todas las columnas, entonces la solución actual es la solución óptima.
        check = of - np.sum(np.reshape(of[list(basis)],(n_rows,1)) * tableau[:,0:n_cols-1],axis=0)
    else:
        # Inicia el algoritmo simplex
        # Calcula cj-zj. Si zj - cj >= 0 para todas las columnas, entonces la solución actual es la solución óptima.
        check = np.sum(np.reshape(of[list(basis)],(n_rows,1)) *tableau[:,0:n_cols-1],axis=0) - of
    count = 0
    while ~np.all(check >=0):
        print(check)
        # Determina la columna pivote: La columna pivotante es la columna correspondiente al mínimo zj-cj.
        pivot_col = np.argmin(check)
        # Determinar los elementos positivos en la columna pivote. Si no hay elementos positivos en la columna pivote, 
        # entonces la solución óptima no tiene límites.
        positive_rows = np.where(tableau[:,pivot_col] > 0)[0]
        if positive_rows.size == 0:
            print('*******SOLUCIÓN SIN LÍMITES******')
            break
        # Determinar la hilera de pivote: prueba de min-ration
        divide=(tableau[positive_rows,n_cols-1]
            /tableau[positive_rows,pivot_col])
        pivot_row = positive_rows[np.where(divide 
            == divide.min())[0][-1]]
        # Actualizar las bases
        basis[pivot_row] = pivot_col
        # Realizar la eliminación gaussiana para hacer girar el elemento uno y los elementos por encima y por debajo del cero:
        tableau[pivot_row,:]=(tableau[pivot_row,:]
            /tableau[pivot_row,pivot_col])
        for row in range(n_rows):
            if row != pivot_row:
                tableau[row,:] = (tableau[row,:] 
                    - tableau[row,pivot_col]*tableau[pivot_row,:])
        if opt =='MIN':
            check = of - np.sum(np.reshape(of[list(basis)],(n_rows,1)) * tableau[:,0:n_cols-1],axis=0)
        else:
            check = np.sum(np.reshape(of[list(basis)],(n_rows,1)) *tableau[:,0:n_cols-1],axis=0) - of
        count += 1
        print('Paso %d' % count)
        print(tableau)
    return basis,tableau

def get_solution(of,basis,tableau):
    # Obtiene el número de columnas de la tableau
    n_cols = tableau.shape[1]
    # Obtener la solución óptima
    solution = np.zeros(of.size)
    solution[list(basis)] = tableau[:,n_cols-1]
    # Determinar el valor óptimo
    value = np.sum(of[list(basis)] * tableau[:,n_cols-1])
    return solution,value

In [12]:
"""
Esta función es muy importante, ya que permite ingresar variables artificiales si son necesarias. Los símbolos que requieren
que se añadan variables artificiales son los signos de >=. El símbolo == requiere una variable artificial en vez de una de 
holgura, el signo >= requiere de una variable artificial 1 y de una variable de holgura -1.
"""

n_b_vector = []
for i in b_vector:
    list1 = [i]
    n_b_vector.append(list1)

#Esta es la matriz que si sirve 
matrix_mat = np.concatenate((mat, n_b_vector),axis =1)
print(matrix_mat)

[[2.82e+02 4.00e+02 1.00e+00 0.00e+00 0.00e+00 2.00e+03]
 [4.00e+00 4.00e+01 0.00e+00 1.00e+00 0.00e+00 1.40e+02]
 [1.00e+00 0.00e+00 0.00e+00 0.00e+00 1.00e+00 5.00e+00]]


In [13]:
print(variable_contraints)

def check_availability(element, collection: iter):
    return element in collection

verificar_menoresque = check_availability('<=', variable_contraints)
print(verificar_menoresque)

verificar_mayoresque = check_availability('>=', variable_contraints)
print(verificar_mayoresque)

verificar_igualesque = check_availability('==', variable_contraints)
print(verificar_igualesque)

['<=', '<=', '<=']
True
False
False


In [14]:
"""
Esta función es utilizado por la maximización, por eso aquí se añaden los coeficientes de la función objetivo, y se añaden
la misma cantidad de ceros que las variables de holgura, por ejemplo si se añadieron 3 variables de holgura, esta identifica
que tiene que añadir 3 ceros. Pero si se añaden variables de holgura y artifiales, la suma de estas será la cantidad de ceros
que se añadan.
"""

matrix_cero = []
size_ceros = num_cols - no_variables
print(size_ceros)

for i in range(size_ceros):
    matrix_cero.append(0)
print(matrix_cero)

objective_matrix = np.concatenate((c_vector, matrix_cero), axis=0)
print(objective_matrix )

3
[0, 0, 0]
[10. 15.  0.  0.  0.]


In [15]:
"""
Esta función la hemos colocado, ya que nuestro algoritmo requiere que se le indican las variables de holgura, las cuales 
son la base de la resolución del caso. Este array con bases es importante para el buen funcionamiento de los pivotes,
ya que si no se le dice explícitamente cuáles son las bases, sería difícil para nuestro algoritmo determinar cuál columna
sale y cuál fila entra.
"""

array_con_bases = []
numero_base = no_variables
for item in range(no_restricciones):  
    array_con_bases.append(item + numero_base)    
print(array_con_bases)

[2, 3, 4]


In [16]:
"""
Esta es la función de maximización. Nos hemos fijado que la maximización requieres que los coeficientes de la función objetivo
no sean negativos, por eso se le manda explícitamente a la función simplex la palabra 'MAX' que identifica la maximización.
Eso permite que se diferencie la minimización con la maximización, y haga el procedimiento adecuado según lo requiera
cada caso.
"""


# Definir la tableau:
tableau = np.array(matrix_mat)
print(tableau)
# Define la función objetivo y las bases iniciales
print(objective_matrix)
    
of = np.array(objective_matrix)
# bases iniciales
#basis = np.array([4,5,6])
basis = np.array(array_con_bases)
# Correr el algorithm simplex
basis,tableau = simplex(of,basis,tableau,'MAX')
# Obtener la solución óptima
optimal_solution,optimal_value = get_solution(of,basis,tableau)
# Imprimir al tableau final.
print('La base final es:')
print(basis)
print('solución')
for i in range(len(optimal_solution)):
    print('X%d'%(i+1),'=',optimal_solution[i])
print('Z=',optimal_value)
#max_function()

[[2.82e+02 4.00e+02 1.00e+00 0.00e+00 0.00e+00 2.00e+03]
 [4.00e+00 4.00e+01 0.00e+00 1.00e+00 0.00e+00 1.40e+02]
 [1.00e+00 0.00e+00 0.00e+00 0.00e+00 1.00e+00 5.00e+00]]
[10. 15.  0.  0.  0.]
[-10. -15.   0.   0.   0.]
Paso 1
[[ 2.42e+02  0.00e+00  1.00e+00 -1.00e+01  0.00e+00  6.00e+02]
 [ 1.00e-01  1.00e+00  0.00e+00  2.50e-02  0.00e+00  3.50e+00]
 [ 1.00e+00  0.00e+00  0.00e+00  0.00e+00  1.00e+00  5.00e+00]]
[-8.5    0.     0.     0.375  0.   ]
Paso 2
[[ 1.00000000e+00  0.00000000e+00  4.13223140e-03 -4.13223140e-02
   0.00000000e+00  2.47933884e+00]
 [ 0.00000000e+00  1.00000000e+00 -4.13223140e-04  2.91322314e-02
   0.00000000e+00  3.25206612e+00]
 [ 0.00000000e+00  0.00000000e+00 -4.13223140e-03  4.13223140e-02
   1.00000000e+00  2.52066116e+00]]
La base final es:
[0 1 4]
solución
X1 = 2.479338842975207
X2 = 3.2520661157024793
X3 = 0.0
X4 = 0.0
X5 = 2.520661157024793
Z= 73.57438016528926


"""
Debido a que solo se harán maximizaciones, dejamos solo el código para realizar estas

Al mismo tiempo, imprime la la solución optimizada.
"""

print("==== MAXIMIZACION ====")
max_function()

In [17]:
"""
Se guarda un array con las soluciones aproximadas de las variables x de la solución óptima. A través de un for
se recorre según el número de variables, para que se garantice que todas las variables x hayan pasado por la prueba.
"""
round_sol = []
posible_sol = []

for i in range(no_variables):
    round_sol.append(round(optimal_solution[i]))



#Soluciones aproximadas 
print(round_sol)

#Multiplicar con la objetivo, las respuestas para hacer pruebas

#Aquí está la Funcion objetivo
print(c_vector)



[2.0, 3.0]
[10.0, 15.0]


In [18]:
"""
Esta es una función que recibe el vector c 'c_vector' y la solución que se encontró. Tiene una variable temp inicializada
en cero, para luego recorrer con for el rango del vector c. Dentro del for, se hace una operación para que el número dentro
de vector_c[i] se multiplique con el número en sol[i], este resultado debe sumarse en la variable temp y la función
retorna el valor de la variable temp.
"""

#Valores optimos funcionales

def posible_funct_val(c_vector, sol):
    temp=0
    for i in range(len(c_vector)):
        temp += c_vector[i]*sol[i]
    return temp

In [19]:
"""
Se definió una función pqcclr que recibe como parámetros a_matrix2, punto, variable_constraints y el b_vector.
básicamente lo que hace es que toma el número de restricciones para hacer un for, que dentro de sí contiene otro for
que se repite según el número de variables. Lo que hace es que suma a la variable temp inicializada en cero, sumarle el 
producto de a_matrix2 en la posición [i][j] por el punto [j]. 

Se hacen unas comprobaciones para saber si el constraint es <=, >= o ==.
    Si se encuentra el constraint <= y la variable temporal es > que b_b_vector[i], devuelve falso.
    Si se encuentra el constraint >= y la variable temporal es < que b_b_vector[i], devuelve falso.
    Si se encuentra el constraint == y la variable temporal es != que b_b_vector[i], devuelve falso.
    Si ninguna de las anteriores se cumple, devuelve true.
    
    Devolver true, significa que cumple con las restricciones, de lo contrario no cumple con las restricciones.
"""

#punto que cumplen con las restricciones
def pqcclr(a_matrix2, punto, variable_contraints, b_vector):
    for i in range(no_restricciones):
        temp = 0
        for j in range(no_variables):
            temp += a_matrix2[i][j]*punto[j]
        if variable_contraints[i]=='<=' and temp > b_vector[i]:
            return False
        if variable_contraints[i]=='>=' and temp < b_vector[i]:
            return False
        if variable_contraints[i]=='==' and temp != b_vector[i]:
            return False
    return True
        
        

In [20]:
"""
Se definió esta función para verificar si los puntos y las variables x aproximadas a enteros proporcionan una solución
óptima y que cumpla con las restricciones necesarias. Se prueban cada una de x aproximadas y se hace uso de la función 
pqcclr realizada anteriormente para verificar que los puntos sigan estando dentro de las restricciones.

Al final, imprime las soluciones óptimas.
"""
import re 

def mvp(c_vector,round_sol,a_matrix2, variable_contraints):
    valores=[]
    puntos=[]
    rangos_variables=[]
    for i in round_sol:
        temp=[]
        for j in range(0, int(i+(2*i))):
            temp.append(j)
        rangos_variables.append(temp)

    #print(rangos_variables)
    for i in range(no_variables):
        print("Rango para variable x", i+1, ": ",rangos_variables[i])
        
    for i in itertools.product(*rangos_variables):
        combinacion=[]
        for j in i:
            combinacion.append(j)
        #print(combinacion)
        if(pqcclr(a_matrix2, combinacion, variable_contraints, b_vector)):
            puntos.append(combinacion)
            valores.append(posible_funct_val(c_vector,combinacion))


    greater=valores[0]
    for item in range(0, len(valores)):
        if valores[item]>greater:
            greater=valores[item]
            grater_index = valores.index(valores[item])        
    #print(grater_index)
    
    #print(puntos[grater_index])
            
        
    print("Z óptimo entero: ", max(valores))    
    print("Solución con variables enteras: ", puntos[grater_index])

In [21]:
"""
A través de un contador se verifica si las variables x óptimas son enteras. Si todas son enteras, se detiene porque el 
resultado ya es entero, de lo contrario tiene que continuar con el programa donde se operarán las variables no
enteras.
"""

#Revisar si la solución es entera
cont_T = 0

for i in range (no_variables):
    check_int = isinstance(optimal_solution[i], int)
    print(check_int)
    if check_int == True:
        cont_T = cont_T + 1
        
if cont_T == no_variables:
    print("Todas las respuestas son enteros")
    sys.exit(1)
else:
    print("No todas las respuestas son enteras, proceda")
    print("--------Respuesta con soluciones enteras--------")
    mvp(c_vector,round_sol,a_matrix2, variable_contraints)

False
False
No todas las respuestas son enteras, proceda
--------Respuesta con soluciones enteras--------
Rango para variable x 1 :  [0, 1, 2, 3, 4, 5]
Rango para variable x 2 :  [0, 1, 2, 3, 4, 5, 6, 7, 8]
Z óptimo entero:  70.0
Solución con variables enteras:  [4, 2]
